# Local Functions EDA

In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

project_root = Path().cwd().parent
sys.path.insert(0, str(project_root))
from src.displayData import inspect_mat_file


In [ ]:
mat_path = project_root / 'tests' / 'testData' / 'LocalFunctions' / 'u1001s0001_sg0001.mat'
features = inspect_mat_file(mat_path)
mat = features.get('localFunctions')
if mat is not None:
    print('Matrix shape:', mat.shape)
    plt.figure()
    plt.plot(mat[:,0], mat[:,1])
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('First two coordinates')
    plt.show()
else:
    print('File not found or data missing')
